In [1]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

def read_and_decode(filename, reshape_dims):
  # Read the file
  img = tf.io.read_file(filename)
  # Convert the compressed string to a 3D uint8 tensor.
  img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # Resize the image to the desired size.
  return tf.image.resize(img, reshape_dims)

CLASS_NAMES = [item.numpy().decode("utf-8") for item in 
               tf.strings.regex_replace(
                 tf.io.gfile.glob(r"E:\\computer vision\\practical-cv\\dataset\\train/*"),
                 r'E:\\computer vision\\practical-cv\\dataset\\train/', "")]
CLASS_NAMES = [item for item in CLASS_NAMES if item.find(".") == -1]
print("These are the available classes:", CLASS_NAMES)



These are the available classes: ['E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\daisy', 'E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\dandelion', 'E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\rose', 'E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\sunflower', 'E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\tulip']


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  # label = tf.math.equal(CLASS_NAMES, label_string)
  return img, label_string

dataset = (tf.data.TextLineDataset(
    "E:\\computer vision\\practical-cv\\dataset\\train_csv.csv").map(decode_csv))

for img, label in dataset.take(1):
  avg = tf.math.reduce_mean(img, axis=[0, 1]) # average pixel in the image
  print(label, avg)

tf.Tensor(b'daisy', shape=(), dtype=string) tf.Tensor([0.6474569  0.6199989  0.50390774], shape=(3,), dtype=float32)


In [3]:
def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
  return img, label

In [4]:
train_dataset = (tf.data.TextLineDataset(
    "E:\\computer vision\\practical-cv\\dataset\\train_csv.csv").
    map(decode_csv)).batch(3)

In [5]:
test_dataset = (tf.data.TextLineDataset(
    "E:\\computer vision\\practical-cv\\dataset\\val.csv").
    map(decode_csv)).batch(3)

In [6]:
"""
in this function i basiclly defining our nural network with learning rate and regulraization 
"""

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

def train_and_evaluate(batch_size = 32,
                       lrate = 0.001, # default in Adam constructor
                       l1 = 0,
                       l2 = 0,
                       num_hidden = 128):
    regulrizer = tf.keras.regularizers.l1_l2(l1, l2),

    train_dataset = (tf.data.TextLineDataset(
    "E:\\computer vision\\practical-cv\\dataset\\train_csv.csv").
    map(decode_csv)).batch(batch_size),

    test_dataset = (tf.data.TextLineDataset(
    "E:\\computer vision\\practical-cv\\dataset\\val.csv").
    map(decode_csv)).batch(batch_size),

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape =  (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        tf.keras.layers.Dense(num_hidden, kernel_regularizer= regulrizer, activation= tf.keras.activations.relu),
        tf.keras.layers.Dense(len(CLASS_NAMES), kernel_regularizer= regulrizer, activation='softmax')
    ])

    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=lrate),
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    history = model.fit(train_dataset, validation_data=test_dataset, epochs = 2)

    return model